# Zakupki website scraping for Piotr

The aim of this notebook is to scrape details of each contract hosted on the Russian Zakupki public sector contract awarding website.

The input for this project will be the Zakupki URL. This code can be run on different dates to pull fresh contract data.

Method:
1.   Identify the number of pages of contracts to be scraped (using the contract filters provided).
2.   Iterate through each page, scraping the registration number of each contract.
3.   Access the website for each contract by placing the registraion number in the URL.
4.   Scrape the details for each contract and add them to a list of Contracts dataclasses.
5.   Format these Contract objects as a dataframe and output the dataframe to a csv file.


The output of this project will be the CSV file, with each row representing a new contract from the webstie.


### Section 1: Setup

In [24]:
import requests
from requests.adapters import HTTPAdapter
from urllib.parse import urlparse, parse_qs
from requests.packages.urllib3.util.retry import Retry
from bs4 import BeautifulSoup
from datetime import date
from dataclasses import dataclass
from tqdm import tqdm
from dateutil import parser
from threading import Thread
import pandas as pd
from datetime import datetime, date, timedelta
import logging
import http.client
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor, as_completed
import math
from os import walk
import json
import numpy as np
import csv


In [25]:
## Finding the memory leak

from collections import Counter
import linecache
import os
import tracemalloc

def display_top(snapshot, key_type='lineno', limit=3):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        # replace "/path/to/module/file.py" with "module/file.py"
        filename = os.sep.join(frame.filename.split(os.sep)[-2:])
        print("#%s: %s:%s: %.1f KiB"
              % (index, filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))


tracemalloc.start()

In [26]:
logging = False

if logging:

    http.client.HTTPConnection.debuglevel = 1

    # You must initialize logging, otherwise you'll not see debug output.
    logging.basicConfig()
    logging.getLogger().setLevel(logging.DEBUG)
    requests_log = logging.getLogger("requests.packages.urllib3")
    requests_log.setLevel(logging.DEBUG)
    requests_log.propagate = True

### Section 2: Determine Number of pages to scrape
Test connection to the website and determine number of pages to scrape

In [27]:
session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

# @lru_cache(maxsize=None)
def getPage(tempURL):
  # If User-Agent is not set to custom, the website will know a Python script is accessing it and block some of the request

  response = session.get(tempURL, headers={'User-Agent': 'Custom'})
  return BeautifulSoup(response.content, "html.parser")

In [28]:
# Getting the dates we want to scrape.

url="https://zakupki.gov.ru/epz/contractfz223/search/results.html?morphology=on&sortBy=BY_UPDATE_DATE&pageNumber={}&sortDirection=false&recordsPerPage=_500&showLotsInfoHidden=false&statuses_0=on&statuses_1=on&statuses=0%2C1&priceFrom=1000000&currencyId=-1&contract223DateFrom={}&contract223DateTo={}&customerPlace=5277383"

startDate = date(2016, 1, 1)
endDate = date(2016, 1, 31)
days = timedelta(days=1)

startDateBefore = startDate

calendar=[]

while startDate<=endDate:
  calendar.append(startDate.strftime('%d.%m.%Y'))
  startDate+=days

print("Created {} dates".format(len(calendar)))
startDate = date(2016, 1, 1)
endDate = date(2016, 1, 31)
days = timedelta(days=1)

startDateBefore = startDate

calendar=[]

while startDate<=endDate:
  calendar.append(startDate.strftime('%d.%m.%Y'))
  startDate+=days

print("Created {} dates".format(len(calendar)))

Created 31 dates
Created 31 dates



### Section 3: Scrape each registration number

Scrape the reg numbers of each contract, so they can be accessed individually

In [29]:
class Page:

    def __init__(self, day, pageNum, pagefile):

        self.day = day
        self.pageNum = pageNum
        self.pagefile = pagefile


In [30]:
def getContracts(page):
#url='https://zakupki.gov.ru/epz/contractfz223/search/results.html?morphology=on&sortBy=BY_UPDATE_DATE&pageNumber=1&sortDirection=false&recordsPerPage=_10&showLotsInfoHidden=false&statuses_0=on&statuses_1=on&statuses=0%2C1&priceFrom=1000000&currencyId=-1&contract223DateFrom=01.01.2019&contract223DateTo=31.01.2019&customerPlace=5277383'
#soup=getPage(url)

 
    # Obtain a list of all the sections of HTML containing a contract in the web page
    listOfContracts = page.find_all("div", {"class": "registry-entry__header-mid__number"})
    regNumbersList=[]

    # Segment the registration number from the URL of each contract - amended for 223, as unique ID was only available in href
    for contract in listOfContracts:
            href = contract.find("a")['href'].split("?id=")[1]
            regNumbersList.append(href)
    print(regNumbersList)

In [31]:
def progress(idx, data):

    x_ = int(((idx+1) * 100) / len(data))
    y_ = idx % math.ceil(len(data) / 10)
    
    print(" ----\n{}% completed\n----".format(x_)) if y_ == 0 else None

In [32]:
# Getting the web page for all the contracts for each date in the range we want to scrape.

regNumbersDict = {}

def getRegNumbersForDate(i, day):

  if day in regNumbersDict:
    return

  tempURL = url.format(day, day, 1)

  print(tempURL)

  page = getPage(tempURL)

  # Scrape the max number of pages
  try:
    maxPageNum = int(page.select('a[data-pagenumber]')[-2].find("span").text)
    print("{} pages for this day".format(maxPageNum))
  except:
    maxPageNum = 1


  # Leave my variable names alone :(
  totalRegNumbersForThisDay = 0

  for i in range(1, maxPageNum+1):

    # Creating a temporary URL for each page containing contracts
    tempPageURL = url.format(day, day, i)

    # Request the page and format it as a BeautifulSoup object so that we can perform scrapings
    page = getPage(tempPageURL)

    regNumbersList = getContracts(page)

    totalRegNumbersForThisDay += len(regNumbersList)

    regNumbersDict[day] = regNumbersList


  print("Fetched day {} had {} contracts \n".format(day, totalRegNumbersForThisDay), end='')

  del page
  del regNumbersList

  progress(i, calendar)


In [34]:

## This now has regNumbers caching too 

cachedRegNums223 = {}

# load the data from the json file
with open('cachedRegNums223.json', 'r') as f:
  cachedRegNums223 = json.load(f)


with ThreadPoolExecutor(max_workers=50) as ex:
  for i, day in enumerate(calendar):
    if day in cachedRegNums223:
      regNumbersDict[day] = cachedRegNums223[day]
      print("Cached day {} had {} contracts \n".format(day, len(regNumbersDict[day])), end='')
    else:
      ex.submit(getRegNumbersForDate, i, day)

    
combinedRegNumbersDict = {**cachedRegNums223, **regNumbersDict}

# print(combinedRegNumbersDict)


with open('cachedRegNums223.json', 'w') as f:
  json.dump(combinedRegNumbersDict, f)


tempRegNumbers = list(regNumbersDict.values())

regNumbers = []

for t in tempRegNumbers:
  regNumbers.extend(t)

# print(regNumbers)

print("------------------- \n {} contracts found in total".format(len(regNumbers)))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Section 4: Details scraping

The Contract Dataclass will store the information during scraping.
If any information can't be scraped, default values have been provided in their place

In [ ]:
@dataclass
class Contract:

  # TODO: Add reg number to class

  # Main Section
  id: float = 0
  price: float = 0.0
  signed: date = None
  deadline: date = None

  # Tab 1
  method: str = "none"
  procurer: str = "none"
  supplier: str = "none"
  proinn: str = "none"
  supinn: str = "none"
  registered: date = None
  address: str = "none"
  number: str = "none"
  mail: str = "none"

  # Tab 2
  code: float = 0.0
  product: str = "none"
  

  def __repr__(self):
    return "\nContract id= {} \n First tab: price={}, signed={}, deadline={}, method={}, procurer={}, supplier={}, proinn={}, supinn={}, registered={}, address={}, number={}, mail={} \n  Second tab: code={}, product={})".format(self.id, self.price, self.signed, self.deadline, self.method, self.procurer, self.supplier, self.proinn, self.supinn, self.registered, self.address, self.number, self.mail, self.code,  self.product)

  

Method for scraping the data from each contract

In [ ]:
def getSectionDict(page):

    sections=page.findAll("section",{"class":"blockInfo__section section"})

    # print([key.findAll("span") for key in sections])

    # Turning the sections into a dictionary that will be easier to work with.
    pairs = [key.findAll("span") for key in sections]

    pairs = list(filter(None, pairs))


    titles = []
    values = []

    for x in pairs:
        if len(x) > 1:
            try:
                titles.append(x[0])
                values.append(x[1])
            except:
                pass

    sectionDict = {titles[i].text.strip() : values[i].text.strip() for i in range(len(titles))}

    return sectionDict


def getTableDict(page, secondTab=False):


    if secondTab:
        sectionOfInterest = page.findAll("div", {"class": "col"})[-1]
    else:
        sectionOfInterest = page

    table = sectionOfInterest.findAll("tr",{"class":"tableBlock__row"})

    # print(table[3])

    headers = [i.text.strip() for i in table[0].findAll("th", {"class":"tableBlock__col tableBlock__col_header"})]
    data = [list(filter(None, [j.strip() for j in i.text.split("\n")])) for i in table[1].findAll("td")]

    if len(headers) == 0:
        headers = [i.text.strip() for i in table[2].findAll("th", {"class":"tableBlock__col tableBlock__col_header"})]
        data = [list(filter(None, [j.strip() for j in i.text.split("\n")])) for i in table[3].findAll("td")]

    if len(headers) == 0:
        headers = [i.text.strip() for i in table[3].findAll("th", {"class":"tableBlock__col tableBlock__col_header"})]
        data = [list(filter(None, [j.strip() for j in i.text.split("\n")])) for i in table[4].findAll("td")]



    # This is hacky.
    if len(data) < len(headers):
        data = [[[]] for i in range(len(headers))]

        

    tableDict = {headers[i] : data[i] for i in range(len(headers))}

    return tableDict

In [ ]:
def scrapeData(reg):

  try:
    # Input: reg = one registration number.

    # Different URL from the one above, this accesses more information from Zakupki.
    dir="https://zakupki.gov.ru/epz/contractfz223/card/{}.html?id={}"
    #"https://zakupki.gov.ru/epz/contract/contractCard/{}.html?reestrNumber={}"

    # Getting the web page for the given contract
    tempDir = dir.format("contract-info", reg)
    page = getPage(tempDir)

    # We probably don't need this with the method I've used below.
    contractTypeTwo = False

    # Enter the text here that should be present to signify the second type of contract.
    if page.findAll(text="Основание заключения контракта с единственным поставщиком"):
      contractTypeTwo = True
      
    id = reg  
    sectionDict = getSectionDict(page)
    firstTableDict = getTableDict(page)

    # print(sectionDict)
    # print(firstTableDict)

    # print(sectionDict, firstTableDict)
    # try:
    #   price=sectionDict["Цена контракта"].replace("\xa0","").replace(",",".").replace("₽","").strip().split()[0]
    # except:
    #   price=sectionDict["Ориентировочное значение цены контракта"].replace("\xa0","").replace(",",".").replace("₽","").strip().split()[0]
    #   try:
    #     price=sectionDict["Максимальное значение цены контракта"].replace("\xa0","").replace(",",".").replace("₽","").strip().split()[0]
    #   except:
    price=page.find('span', {'class':'cardMainInfo__content cost'}).text.strip().replace("₽","").replace(" ","")
      
    signed=sectionDict["Дата заключения контракта"].split()[0]
    deadline=sectionDict["Дата окончания исполнения контракта"].split()[0]
    
    ### fixed issue with method ### 
    try:
      method = sectionDict["Способ определения поставщика (подрядчика, исполнителя)"]
    except:
      if page.findAll(text="Основание заключения контракта с единственным поставщиком"):
            method="Закупка у единственного поставщика (подрядчика, исполнителя)"
            
    procurer=sectionDict["Полное наименование заказчика"]
    supplier=firstTableDict["Организация"][0]

    proinn=sectionDict["ИНН"]

    ### fixed issues for missing values sometimes in the table ###
    
    registered=firstTableDict["Организация"][-1]
    
    ### fixed, testing ###   
    try: 
      if firstTableDict["Организация"][-4]=="КПП:":
          supinn=firstTableDict["Организация"][-5]
      else:
          supinn=firstTableDict["Организация"][-3]
    except:
      supinn=""

    ### fixed issues in lower table ### 
    
    try:
      address=firstTableDict["Адрес места нахождения"]
    except:
      address=firstTableDict["Адрес в стране регистрации"]
    number=firstTableDict["Телефон, электронная почта"][0]
    mail=firstTableDict["Телефон, электронная почта"][1]

    ### details about winner - ALSO, THERE'S OPTION TO SCRAPE SUBCONTRACTORS ### 

    page.decompose()

    ### Second tab ###
    tempDir = dir.format("payment-info-and-target-of-order", reg)
    page = getPage(tempDir)

    ### code stands for the product code, which can be later identified to return industry type ### 
    
    secondTableDict = getTableDict(page, True)
    
    try:
      code = secondTableDict["Позиции по КТРУ, ОКПД2"][1][1:-1]
    except:
      code = ''
        
    product = secondTableDict["Наименование объекта закупки и его характеристики"][0]
    
    # Create the Contract dataclass object and append it to a list of objects.
    # This method means that missing data can be accounted for.
    # print(method)

    contract = Contract(id=id, price=price, signed=signed, deadline=deadline, method=method, procurer=procurer, supplier=supplier, proinn=proinn, supinn=supinn, registered=registered, address=address, number=number, mail=mail, code=code, product=product)
    
    # contracts.append(contract)
    # print('Completed {}'.format(id))

    page.decompose()

    return contract
  except Exception as e:
    failedRegNumbers.append(reg)
    print("Failed to scrape {}".format(reg))
    print(e)

### Section 5: Starting execution
Scrape the contracts themselves using threading

In [ ]:
def scrape(reg):
    
    try:
        _ = int(reg)
        # print("Scraping {}".format(reg))
        return [scrapeData(reg)]
    except TypeError:
        
        # TODO make 500 contracts change here.
        
        contracts = []

        

        for idx, r in enumerate(reg):
            # print("Scraping {}".format(r))
            contracts.append(scrapeData(r))
            # progress(i + idx, regNumbers)

        return contracts

In [ ]:
progressNum = 0

failedRegNumbers = []

# regNumbers = [3666404326716000002, 3666404326716000002, 1722300731616000034, 1722300731616000034, 2666001041516000469]

print("Starting scrape with {} reg numbers\n".format(len(regNumbers)))

# scrape(regNumbers[:10])

# for contract in contracts:
#     print(contract)

# for regNumber in tqdm(regNumbers[:50]):
#   thread = Thread(target = scrapeData, args = (regNumber,))
#   thread.start()

# regNumbers = ['3662502457421000001']
# regNumbers = ['1665800691921000016']

threading = True

if threading:

    interval = 1

    with ThreadPoolExecutor(max_workers=20) as ex:
        threads = []

        cachedContracts = {}

        with open('cachedContracts.csv', encoding="utf-8") as f:
            cachedContracts = list(csv.reader(f))
        
        cachedContracts = list(filter(None, cachedContracts))
        
        cachedContractRegNums = [row[0] for row in cachedContracts]

        uncachedRegNumbers = list(set(regNumbers) - set(cachedContractRegNums))

        print("{} of {} contracts are uncached. Fetching...".format(len(uncachedRegNumbers), len(regNumbers)))
        
        for i in range(0, len(uncachedRegNumbers), interval):
            tempNumbers = uncachedRegNumbers[i:i+interval]
            # print(tempNumbers)
            threads.append(ex.submit(scrape, tempNumbers))
        

        completed = 0

        for result in tqdm(as_completed(threads)):

            try:
                contracts = result.result()

                completed += interval

                # progress(completed, uncachedRegNumbers)

                formattedContracts = [list(contract.__dict__.values()) for contract in contracts]

                # newCachedContracts = {**cachedContracts, **formattedContracts}

                with open('cachedContracts.csv', 'a', encoding="utf-8", newline='') as f:
                    writer = csv.writer(f)
                    writer.writerows(formattedContracts)
            except Exception as e:
                print(e)
                
            # print("{} finished".format(interval))

else:

    scrape(regNumbers)

# print("Scraped {} contracts".format(len(contracts)))
print("Failed to scrape {} contracts".format(len(failedRegNumbers)))
print(failedRegNumbers) if len(failedRegNumbers) > 0 else None

snapshot = tracemalloc.take_snapshot()
display_top(snapshot)

Starting scrape with 644 reg numbers

0 of 644 contracts are uncached. Fetching...


0it [00:00, ?it/s]

Failed to scrape 0 contracts


Top 3 lines
#1: ipykernel_17876\665796888.py:33: 77997.6 KiB
    cachedContracts = list(csv.reader(f))
#2: json\decoder.py:353: 4465.9 KiB
    obj, end = self.scan_once(s, idx)
#3: ipykernel_17876\665796888.py:37: 342.8 KiB
    cachedContractRegNums = [row[0] for row in cachedContracts]
606 other: 955.0 KiB
Total allocated size: 83761.3 KiB


### Now that the data is saved to the hard disk, we can run the below code without needing to rerun the scraping process

In [ ]:
cachedContracts = []

with open('cachedContracts.csv', encoding="utf-8") as f:
    cachedContracts = list(csv.reader(f))


df = pd.DataFrame(columns=Contract().__dict__.keys(), data=cachedContracts)

df['signed'] = pd.to_datetime(df['signed'])
df['deadline'] = pd.to_datetime(df['deadline'])

start = datetime.fromordinal(startDateBefore.toordinal()).strftime("%Y-%m-%d")
end = datetime.fromordinal(endDate.toordinal()).strftime("%Y-%m-%d")

print(start, end)

mask = (df['signed'] >= start) & (df['signed'] <= end)

selectedDatesDF = df.loc[mask]

selectedDatesDF.head()

c:\Users\majda\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '30.07.2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\majda\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '13.09.2017' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\majda\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '29.03.2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\majda\AppData\Local\Programs\Python\Python310\lib\site-packag

2016-01-01 2016-01-31


,id,price,signed,deadline,method,procurer,supplier,proinn,supinn,registered,address,number,mail,code,product
61,1666101454016000286,"2 950 338,83",2016-01-11,2016-12-22,Электронный аукцион,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИ...,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АПЕЙ...",,6679050080,07.05.2014,"['620089, обл СВЕРДЛОВСКАЯ 66, г ЕКАТЕРИНБУРГ,...",7-343-2195431,apeiron2014@mail.ru,32.50.50.000,Поставка расходных материалов
132,2663200681916000001,"2 499 776,00",2016-01-29,2016-12-31,Закупка у единственного поставщика (подрядчика...,государственное бюджетное профессиональное обр...,"ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""ЭНЕРГОСБЫТ ПЛЮС...",6632006819,5612042824,22.05.2014,"['143421, обл МОСКОВСКАЯ 50, р-н КРАСНОГОРСКИЙ...",7-34385-60792,sesb@ies-holding.com,35.13.10.000,Поставка электрической энергии
134,3663900278916000001,"1 240 700,00",2016-01-25,2016-12-31,Закупка у единственного поставщика (подрядчика...,МУНИЦИПАЛЬНОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ У...,"МУНИЦИПАЛЬНОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ ""НОВЫЕ ТЕХ...",6639002789,6683008067,10.02.2015,"['624030, обл СВЕРДЛОВСКАЯ 66, р-н БЕЛОЯРСКИЙ,...",343-77-74928,Beloyarsk@rcurala.ru,35.30.11.119,Коммунальные услуги - теплоснабжение
182,3662400708616000005,"1 284 297,19",2016-01-03,2016-12-31,Электронный аукцион,МУНИЦИПАЛЬНОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ УЧ...,Килина Елена ЮрьевнаИндивидуальный предпринима...,6624007086,662400105623,15.03.2011,"['624221 Свердловская обл., г. Нижняя Тура, ул...",7-912-6678681,kilina62@mail.ru,56.29.20.120,Организация услуги по организации горячего пит...
317,3661201544516000008,"2 041 652,88",2016-01-25,2016-12-31,Электронный аукцион,"муниципальное казенное учреждение ""Управление ...",ГРИГОРЬЕВ ВЛАДИМИР АЛЕКСАНДРОВИЧИндивидуальный...,6612015445,661203005272,19.11.2013,"['623412, ОБЛ СВЕРДЛОВСКАЯ 66, Г КАМЕНСК-УРАЛЬ...",7-912-2550099,v.g.metresurs@mail.ru,81.30.10.000,"Содержание газонов, цветников, зеленых насажде..."


In [ ]:
selectedDatesDF.shape

(780, 15)

### Section 6: Output

Convert the list of contract classes to a dataframe so that they can be exported to a csv file

In [ ]:
selectedDatesDF.to_csv("zakupki{}to{}.csv".format(startDateBefore, endDate))